In [16]:
from PIL import Image
import numpy as np
import math
import random
import pandas as pd
import webcolors
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

FOLDER_PATH = 'data/'

In [17]:
def load_image(filename: str, resize: tuple):
  img = Image.open(filename)
  img = img.resize(resize)
  return np.array(img).astype(np.float64)

In [18]:
def show_image(filename: str):
  img = Image.open(filename)
  img.show()

In [19]:
def euclidean_distance(p1, p2):
    return math.sqrt(sum((px - qx) ** 2.0 for px, qx in zip(p1, p2)))

In [20]:
def manhattan_distance(p1, p2):
  return sum(abs(px - qx) for px, qx in zip(p1, p2))

In [21]:
def nearest_centroid(point, centroids, distance):
  flag = False
  idx_centroid, min_distance = 0, 0

  for i in range(len(centroids)):
    current_distance = distance(point, centroids[i])
    if not flag or current_distance <  min_distance:
      flag = True
      
      idx_centroid = i
      min_distance = current_distance

  return idx_centroid, min_distance

In [22]:
def lloyd(data, k, iters, type, distance):
  distance_func = euclidean_distance if distance == "euclidian" else manhattan_distance

  go = True
  epoch_c = 0
  error = 0

  cluster_array = np.zeros(len(data))

  centroids = random.choices(data, k=k)

  while go:
    for element, i in zip(data, range(len(cluster_array))):
      idx_centroid, min_distace = nearest_centroid(
        element, centroids, distance_func)

      error += min_distace
      cluster_array[i] = idx_centroid

    for i in range(k):
      cluster_average = []
      for element, j in zip(data, range(len(cluster_array))):
        if cluster_array[j] == i:
          cluster_average.append(element)

      centroids[i] = np.average(cluster_average, axis=0)
      
    epoch_c += 1
    go = False if epoch_c > iters else True

    return centroids, error



In [23]:
def closest_colour(requested_colour):
    min_colours = {}
    for key, name in webcolors.CSS3_HEX_TO_NAMES.items():
        r_c, g_c, b_c = webcolors.hex_to_rgb(key)
        rd = (r_c - requested_colour[0]) ** 2
        gd = (g_c - requested_colour[1]) ** 2
        bd = (b_c - requested_colour[2]) ** 2
        min_colours[(rd + gd + bd)] = name
    return min_colours[min(min_colours.keys())]

In [24]:
def get_colour_name(requested_colour):
    try:
        closest_name = actual_name = webcolors.rgb_to_name(requested_colour)
    except ValueError:
        closest_name = closest_colour(requested_colour)
        actual_name = None
    return actual_name, closest_name

In [25]:
img = "img_01.png"
show_image(FOLDER_PATH + img)

img_array = load_image(FOLDER_PATH + img, (128, 128))
img_array = img_array.reshape(128*128, 3)

color_palletes, error = lloyd(img_array, 5, 1e5, "", "euclidean")

for color in color_palletes:
  print(get_colour_name(color))

(None, 'chocolate')
(None, 'saddlebrown')
(None, 'maroon')
(None, 'maroon')
(None, 'maroon')


In [26]:
color_palletes

[array([193.2672847 ,  99.40469589,   4.28019407]),
 array([134.47048611,  39.11545139,   2.53559028]),
 array([125.60285375,  33.64803805,   2.35790725]),
 array([93.47763864, 21.69946333,  1.78711986]),
 array([111.6938186 ,  27.32582322,   2.04852686])]

In [27]:
img = "img_02.png"
show_image(FOLDER_PATH + img)

img_array = load_image(FOLDER_PATH + img, (128, 128))
img_array = img_array.reshape(128*128, 3)

color_palletes, error = lloyd(img_array, 5, 1e5, "", "euclidean")

for color in color_palletes:
  print(get_colour_name(color))

(None, 'darkslateblue')
(None, 'rosybrown')
(None, 'darkslategray')
(None, 'darkmagenta')
(None, 'midnightblue')


In [28]:
color_palletes

[array([ 91.36070111,  38.40774908, 102.3302583 ]),
 array([171.26374443, 119.52080238, 177.16976226]),
 array([56.08534107, 24.61538462, 59.34746009]),
 array([131.66546227,  46.80680128, 140.965356  ]),
 array([ 66.52697095,  37.17101363, 101.71161826])]

In [29]:
img = "img_03.png"
show_image(FOLDER_PATH + img)

img_array = load_image(FOLDER_PATH + img, (128, 128))
img_array = img_array.reshape(128*128, 3)

color_palletes, error = lloyd(img_array, 5, 1e5, "", "euclidean")

for color in color_palletes:
  print(get_colour_name(color))

(None, 'silver')
(None, 'sienna')
(None, 'darkolivegreen')
(None, 'rosybrown')
(None, 'gray')


In [30]:
color_palletes


[array([194.06914567, 197.24953903, 176.5414874 ]),
 array([137.94977169, 109.16666667,  68.55707763]),
 array([78.35770578, 64.308669  , 39.6234676 ]),
 array([157.11536851, 153.67650754, 122.75481575]),
 array([133.35189003, 126.10687285,  94.62886598])]

In [31]:
img = "img_04.png"
show_image(FOLDER_PATH + img)

img_array = load_image(FOLDER_PATH + img, (128, 128))
img_array = img_array.reshape(128*128, 3)

color_palletes, error = lloyd(img_array, 5, 1e5, "", "euclidean")

for color in color_palletes:
  print(get_colour_name(color))

(None, 'black')
(None, 'gray')
(None, 'darkslategray')
(None, 'firebrick')
(None, 'tomato')


In [32]:
color_palletes


[array([31.89087302, 14.51785714, 24.7202381 ]),
 array([127.48420381, 119.74460164, 140.27358792]),
 array([68.45902883, 51.01745068, 71.18310572]),
 array([176.9147351 ,  40.17301325,  33.89652318]),
 array([240.06302202,  85.59149582,  65.39028094])]

In [33]:
img = "img_05.png"
show_image(FOLDER_PATH + img)

img_array = load_image(FOLDER_PATH + img, (128, 128))
img_array = img_array.reshape(128*128, 3)

color_palletes, error = lloyd(img_array, 5, 1e5, "", "euclidean")

for color in color_palletes:
  print(get_colour_name(color))

(None, 'dimgray')
(None, 'gray')
(None, 'black')
(None, 'seagreen')
(None, 'darkslategray')


In [34]:
color_palletes


[array([87.15797172, 83.76840566, 87.06923452]),
 array([143.7542153 , 137.80674449, 141.05642023]),
 array([11.93893623, 22.57797573, 29.61425252]),
 array([ 54.04107143, 120.75178571, 109.95178571]),
 array([56.32619844, 58.95295429, 56.01070234])]